## Part 01 : Review

Review the following files:

1) [NS_LECTURE_06.pdf](https://canvas.ucdavis.edu/files/22546365/download?download_frd=1) (especially page 07)
2) [NS_LECTURE_07.pdf](https://canvas.ucdavis.edu/files/22546356/download?download_frd=1) (especially pages 01 & 02).
3) [Professor Saito's Lecture Notes](https://canvas.ucdavis.edu/courses/859953/files/folder/LECTURE_NOTES/PROFESSOR_SAITOS_LECTURE_NOTES_2022_WQ}{PROFESSOR\_SAITOS\_LECTURE\_NOTES) 09 through 11 (inclusive).

## Part 02 : Theory

Let $m$ and $n$ be integers such that $m > n$.

Given $m$ distinct **grid points** in the unit interval,
$$
x_1=0, x_2, ..., x_m=1,
$$
and $m$ **data points**,
$$
y_1, y_2, ..., x_m \in \mathbb{R},
$$

the goal is for you to **fit a polynomial of degree $(n - 1)$**
$$
p(x) = c_0 + c_1 x^1 + c_2 x^2 + ... + c_{n-1} x^{n-1}
$$
to the data $(x_i, y_i), i = 1,2,...,m$.  In this programming exercise the data points $y_i$ are given by
$$
y_i = \cos({6 x_i}).
$$

The polynomial $p(x)$ is the best Least Squares fit or approximation to the data if it minimizes the **square** of the two norm of the residual
$$
\lVert \mathbf{r} \rVert_2^2
\stackrel{\mathrm{def}}{=}
\sum_{i=1}^{m}{|p(x_i)-y_i|^2}.
$$

We can write this requirement in matrix notation in the following way:
$$
\underbrace{\begin{bmatrix}
1      & x_1    & x_1^2  & \cdots & x_1^{n-1} \\
1      & x_2    & x_2^2  & \cdots & x_2^{n-1} \\
\vdots & \ddots & \ddots & \ddots & \vdots    \\
1      & x_m    & x_m^2  & \cdots & x_m^{n-1} \\
\end{bmatrix}}_{A}
\;
\underbrace{\begin{bmatrix}
            c_0    \\
            c_2    \\
            \vdots \\
            c_{n-1}
            \end{bmatrix}}_{\mathbf{c}}
            \; = \;
\underbrace{\begin{bmatrix}
            y_1    \\
            y_2    \\
            \vdots \\
            y_{m}
            \end{bmatrix}}_{\mathbf{y}}.
$$

The matrix $A$ is called a **Vandermonde** matrix.  Thus, the polynomial $p(x)$ is the best Least Squares fit to the data if it minimizes
$$
\lVert \mathbf{r} \rVert_2^2
\stackrel{\mathrm{def}}{=}
\lVert \mathbf{y} - A \mathbf{c} \rVert_2^2,
$$

where $\mathbf{r} = \mathbf{y} - A \mathbf{c}$ is the residual.


## Part 03 : Setup

Make sure you run the following cell **before** moving on to the next cells.  More generally, all code cells in this notebook should be run in order.

You can run a cell by highlighting it and typing `<Shift+Enter>`.

In [1]:
# Run some imports, and increase the default print-precision.

import numpy as np
import pandas as pd

np.set_printoptions(precision=16)
pd.options.display.float_format = '{:,.16f}'.format

### Generate values for our system of equations.

* Take $m = 50$ and $n = 12$.

* Define $\bf{x}$ to be the m-vector (which we will refer to as the **grid**) corresponding to $m$ equally spaced grid points $x_i$ from 0 to 1 (inclusive).
  * You may want to use [np.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) to help you with this.

* Define $\bf{y}$ to be the function $\cos(6x)$ evaluated at the grid points.
  * Be sure to use numpy's [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) for this!
  * For example, to compute the elementwise sin of a vector `vec`, you can write `np.sin(vec)`.
  * One could also write `np.array([np.sin(v_i) for v_i in vec])`, but this is **slow**. Don't do it!

* Define $A$ to be the $m \times n$ **Vandermonde Matrix** associated with least-squares fitting on this grid by a polynomial of order $n-1$.
  * You may want to use [np.vander](https://numpy.org/doc/stable/reference/generated/numpy.vander.html) to help you with this.
  * Take note the optional `increasing` parameter! Writing `A = np.vander(x, 12)` will only get you partial credit.  Make sure to *look* at $A$ and compare it to what you see in "Theory".

**Instructions:**

In the following cell, define `x`, `y`, and `A` in Python as described above.  Make sure to run the cell, as we'll be using these values below.

In [3]:
m = 50
n = 12

x = ???
y = ???
A = ???

## Part 04 : Algorithms

Using the dataset above, you will *approximate* the least squares solution for $\hat{c}$ where $\hat{c}$ is the solution to $A\hat{c} = \bf{y}$, using six different methods.

You will organize your results using a [pandas Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), which is a popular data-structure for storing **tabular** data (ie, spreadsheets).

Ultimately you will produce a $n \times 6$ table, with one **row** for each $\hat{c}_i$ and one **column** for each algorithm.

### 4a) Builtin Solver.

To get you started, we've populated the first column for you (numpy's builtin [lstsq](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html) method).  Make sure to **run** this cell before moving on!

In [23]:
c_hat = pd.DataFrame()
c_hat.index.name = 'i'

# Try removing `rcond=-1` below.
# The warning you get will help you with your writeup!
# A particularly clever person might even add an additional
# column to their report...
c_hat['np.lstsq'] = np.linalg.lstsq(A, y, rcond=-1)[0]
c_hat

,np.lstsq
i,
0,1.0000003170422456
1,-0.0001438869171453
2,-17.9933508474408228
3,-0.1185149457554676
4,55.0934311258083298
5,-5.9507856045636869
6,-44.3932868788175483
7,-45.3474040562426666
8,106.5916940357666363


### 4b) Normal Equations.

Solve for $\hat{c}$ using $A^T A \hat{c} = A^T \mathbf{y}$.  Name your result `"normal"`.

**Note:** you should ***always*** use numpy's [solve](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html) method rather than [inv](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html).  Although they look similar, `np.linalg.solve(A, y)` is a *far* more stable approximation than `np.linalg.inv(A) @ y` (note that `@` is matrix-multiplication for numpy arrays).

In [17]:
c_hat["normal"] = ???

### 4c-4e) QR Decompositions.

Next, solve for $\hat{c}$ given the QR decomposition of `A`, once again using `np.linalg.solve`.

Use your helper to solve for $\hat{c}$ using numpy's builtin QR solver, as well as the `classical_gram_schmidt` and `modified_gram_schmidt` methods you coded up in `hw01`.
* You should paste your functions directly into the corresponding cell.
* We have provided our `classical_gram_schmidt` implementation as an example. Your `modified_gram_schmidt` implementation should have the same **signature** (inputs/outputs).
* You can use `unit_tests/hw01.ipynb` to check your implementations.

Name your results `"qr"`, `"cgs"`, and `"mgs"`, respectively.

In [38]:
Q, R = np.linalg.qr(A)
c_hat["qr"] = ???

In [20]:
def classical_gram_schmidt(A: np.ndarray):
    """Returns the QR decomposition of A using the classical gram-schmidt algorithm.

    Arguments:
        A (np.ndarray): A matrix whose columns are linearly independent.
    Returns:
        (Q, R): The QR decomposition of A.
    """
    A = np.asanyarray(A, dtype=np.float64)
    m, n = A.shape
    assert m >= n

    Q = np.zeros_like(A)
    for j in range(n):
        qj = A[:, j].copy()
        for k in range(j):
            qj -= (Q[:, k] @ A[:, j]) * Q[:, k]
        Q[:, j] = qj / np.linalg.norm(qj)
    return Q, Q.T @ A

Qcgs, Rcgs = classical_gram_schmidt(A)
c_hat["cgs"] = ???

In [4]:
def modified_gram_schmidt(A: np.ndarray):
    """Returns the QR decomposition of A using the modified gram-schmidt algorithm.

    Arguments:
        A (np.ndarray): A matrix whose columns are linearly independent.
    Returns:
        (Q, R): The QR decomposition of A.
    """
    # Paste your solution here!!

Qmgs, Rmgs = modified_gram_schmidt(A)
c_hat["mgs"] = ???

### 4f) SVD.

Use [np.linalg.svd](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html) to approximate $\hat{c}$.  Name your result `"svd"`.

In [ ]:
U, S, Vh = np.linalg.svd(A, full_matrices=False)
c_hat["svd"] = ???

## 5) Report.

### 5a) Print results in Pandas.

Take a second to compare your approximations digit-by-digit.  The values approximated by each algorithm should be *close* but not *identical*.  If one of your solvers looks severely off, this might be a good opportunity to check for bugs.

You may find it easier to look at the **transpose** of your table (`c_hat.T`), such that each **row** is an algorithm and each **column** is a particular $\hat{c}_i$.  Your final report, however, should include algorithms as columns.

In [ ]:
c_hat  # This table will go in your report.
# c_hat.T  # This may be easier to look at.  Uncomment this line to see the transposed view.

,np.lstsq,cgs
i,,
0,1.0000003170422456,1.0000003170422123
1,-0.0001438869171453,-0.0001438868981619
2,-17.9933508474408228,-17.9933508481824376
3,-0.1185149457554676,-0.1185149345670802
4,55.0934311258083298,55.0934310385098840
5,-5.9507856045636869,-5.9507852041438811
6,-44.3932868788175483,-44.3932880303301971
7,-45.3474040562426666,-45.3474019208441206
8,106.5916940357666363,106.5916914843842846


### 5b) Format results with LaTex.

We've provided a `values_to_latex` function, which converts a `np.array` of floats to a Latex string that you can copy-paste into your report. Run this below, and copy-paste the output into your editor of choice.

Identify which digits in each column you believe are correct and which digits are incorrect.  Digits may be incorrect due to one of
* An unstable (direct) solver.
* An ill-conditioned matrix.
* Roundoff error.

Wrap the questional digits in the LaTex `\rd{ }` macro to turn those digits red.

$\def\rd#1{\textcolor{red}{#1}}$

For example, if one of your values were $1.00123$ and you believe the final two digits to be suss, you would write `1.001\rd{23}` to display $1.001\rd{23}$.

**Task:** Put the *annotated* table in the markdown cell below.  Make sure to **run the code cell *after* adding all the columns to `c_hat`**.

**Note:** The command `\rd{...}` is a latex **macro** [defined](http://www.emerson.emory.edu/services/latex/latex_19.html) in this cell.  If you want to use this in a `.tex` file, you can write `\newcommand{rd}[1]{\textcolor{red}{#1}}` in the [preamble](https://olivierpieters.be/blog/2016/08/10/latex-preamble).  We did something *slightly* different here, because jupyter notebooks don't have preambles. For those curious enough to double-click this cell, what you'll find is considered *bad practice*.

In [ ]:
import textwrap

def values_to_latex(df, fmt='{:+0.16f}'):
    """Converts a dataframe of floats to a latex string."""
    if isinstance(fmt, str):
        fmt = fmt.format

    schema = "|".join( "c" * len(df.columns) )
    header = '\t& '.join(
        f'\\mathrm{{{algo}}}'
        for algo in df.columns)
    content = "\t\\\\\n        ".join(
        "\t& ".join(map(fmt, row))
        for row in df.values)
    return textwrap.dedent(f"""
        $$
        \\scriptsize
        \\begin{{array}}{{ |{schema}| }}
        \\hline
        {header}\t\\\\
        \\hline
        {content}\t\\\\
        \\hline
        \\end{{array}}
        $$
        """)

# Run this cell to generate an unstyled latex table.
print(values_to_latex(c_hat))


$$
\scriptsize
\begin{array}{ |c|c| }
\hline
\mathrm{np.lstsq}	& \mathrm{cgs}	\\
\hline
+1.0000003170422456	& +1.0000003170422123	\\
-0.0001438869171453	& -0.0001438868981619	\\
-17.9933508474408228	& -17.9933508481824376	\\
-0.1185149457554676	& -0.1185149345670802	\\
+55.0934311258083298	& +55.0934310385098840	\\
-5.9507856045636869	& -5.9507852041438811	\\
-44.3932868788175483	& -44.3932880303301971	\\
-45.3474040562426666	& -45.3474019208441206	\\
+106.5916940357666363	& +106.5916914843842846	\\
-56.6056926064087023	& -56.6056907094191999	\\
+7.6179961294152880	& +7.6179953310812563	\\
+1.0662278302210748	& +1.0662279754750210	\\
\hline
\end{array}
$$



$$
\text{Put your table here!} \\
\text{(double-click the cell to edit)}
$$


### 5c) Comment and Analyze.

In Professor Saito’s lecture notes referenced above, there are examples showing how stable or unstable the computation can be when you use each of these algorithms. Give a **thoughtful explanation** for what you observe with regards to the accuracy of **each of the six** algorithms that you used to compute the numerical solution of $A\hat{c} = \mathbf{y}$ in parts (4a-4f).

$$
\text{Put your writeup here!} \\
\text{(double-click the cell to edit)}
$$